In [1]:
import os
import shutil
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, VBox, Output, IntText, Label
from IPython.display import display, clear_output

# Disable GPU usage
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Define dataset directories
original_dataset_dir = 'Datasets/the_wildfire_dataset_2n_version'
corrected_dataset_dir = 'Datasets/corrected_wildfire_dataset'
removed_dir = 'Datasets/removed'
changes_dir = 'Datasets/changes'

# Define subdirectories as per new structure
folders = [
    'test/fire/fire', 'test/fire/smoke', 'test/nofire/fire_confounders',
    'test/nofire/smoke_confounders', 'test/nofire/non_fire',
    'train/fire/fire', 'train/fire/smoke', 'train/nofire/fire_confounders',
    'train/nofire/smoke_confounders', 'train/nofire/non_fire',
    'val/fire/fire', 'val/fire/smoke', 'val/nofire/fire_confounders',
    'val/nofire/smoke_confounders', 'val/nofire/non_fire'
]

2024-11-01 16:02:28.601226: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-01 16:02:28.775676: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-01 16:02:28.845591: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-01 16:02:28.863921: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-01 16:02:28.991929: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Create directories if they don't exist
for folder in folders:
    os.makedirs(os.path.join(corrected_dataset_dir, folder), exist_ok=True)
os.makedirs(removed_dir, exist_ok=True)
os.makedirs(changes_dir, exist_ok=True)

# Initialize changes log
changes_log_path = os.path.join(changes_dir, 'changesMade.txt')
if not os.path.exists(changes_log_path):
    with open(changes_log_path, 'w') as f:
        f.write("List of reclassified images:\n")

In [3]:
# Function to read removed images from changesMade.txt
def get_removed_images(changes_log_path):
    removed_images = set()
    if os.path.exists(changes_log_path):
        with open(changes_log_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                line = line.strip()
                if '-> Removed' in line:
                    image_name = line.split(':')[0].strip()
                    removed_images.add(image_name)
    return removed_images

# Function to get all images from a directory
def get_all_images_in_dir(directory):
    image_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                image_paths.append(os.path.join(root, file))
    return image_paths

# Get list of removed images
removed_images = get_removed_images(changes_log_path)

# Get list of images in corrected dataset
corrected_images = set()
corrected_image_paths = get_all_images_in_dir(corrected_dataset_dir)
for img_path in corrected_image_paths:
    corrected_images.add(os.path.basename(img_path))

# Get list of all images in original dataset along with their labels and categories
original_image_info = []  # List of tuples: (img_path, label, category)
for category in ['train', 'test', 'val']:
    for label in ['fire', 'nofire']:
        directory = os.path.join(original_dataset_dir, category, label)
        if os.path.exists(directory):
            for img_name in os.listdir(directory):
                if img_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                    img_path = os.path.join(directory, img_name)
                    original_image_info.append((img_path, label, category))

# Now, identify images that have not been reclassified or removed
img_paths_to_process = []
for img_path, label, category in original_image_info:
    img_name = os.path.basename(img_path)
    if img_name not in corrected_images and img_name not in removed_images:
        img_paths_to_process.append((img_path, label, category))

# Load MobileNetV3 model for feature extraction
model = tf.keras.applications.MobileNetV3Small(weights="imagenet", include_top=True)

# Preprocess image for model input
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.mobilenet_v3.preprocess_input(img_array)
    return img_array

# Generate Grad-CAM heatmap
def get_gradcam_heatmap(model, img_array, last_conv_layer_name="conv_1"):
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
        
    # Compute gradients
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    
    # Create heatmap
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.reduce_max(heatmap)
    return heatmap.numpy()

# Superimpose Grad-CAM heatmap on original image
def superimpose_gradcam(img_path, heatmap, alpha=0.4):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img, alpha, heatmap, 1 - alpha, 0)
    return superimposed_img

# Move misclassified images and record changes
def move_misclassified_image(img_path, corrected_label, corrected_subcategory, category, current_label):
    sub_dir = f"{corrected_label}/{corrected_subcategory}"
    new_path = os.path.join(corrected_dataset_dir, category, sub_dir, os.path.basename(img_path))
    os.makedirs(os.path.dirname(new_path), exist_ok=True)
    shutil.move(img_path, new_path)
    print(f"Moved {img_path} to {new_path}")
    
    # Log the change
    if corrected_label != current_label or corrected_subcategory != current_label:
        with open(changes_log_path, 'a') as f:
            f.write(f"{os.path.basename(img_path)}: {current_label} -> {corrected_label}/{corrected_subcategory}\n")

# Move confusing images to the /removed directory and log this change
def remove_image(img_path, current_label):
    new_path = os.path.join(removed_dir, os.path.basename(img_path))
    shutil.move(img_path, new_path)
    print(f"Moved {img_path} to {new_path} (Removed)")
    with open(changes_log_path, 'a') as f:
        f.write(f"{os.path.basename(img_path)}: {current_label} -> Removed\n")

/home/johan/anaconda3/lib/python3.9/site-packages/keras/src/applications/mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(
2024-11-01 16:02:38.407708: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-01 16:02:38.407789: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:135] retrieving CUDA diagnostic information for host: pop-os
2024-11-01 16:02:38.407793: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:142] hostname: pop-os
2024-11-01 16:02:38.407938: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:166] libcuda reported version is: 560.35.3
2024-11-01 16:02:38.407958: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:170] kernel reported version is: 560.35.3
2024-11-01 16:02:38.407

In [4]:
# Global variables for tracking current image and classification history
classification_history = []
img_idx = 0
img_paths = img_paths_to_process  # Use the images to process

# Display widgets and images
out = Output()
completed_label = Label(value="Images classified: 0")

# Classification function
def classify_image(new_label, subcategory):
    global img_idx
    if img_idx >= len(img_paths):
        print("All images have been reviewed.")
        return
    img_path, current_label, category = img_paths[img_idx]

    # Move image to the corrected directory if reclassified
    if new_label != current_label or subcategory != current_label:
        move_misclassified_image(img_path, new_label, subcategory, category, current_label)
        classification_history.append((img_path, new_label, category, subcategory, current_label))  # Record the action for undo
    
    # Move to the next image
    img_idx += 1
    completed_label.value = f"Images classified: {img_idx}"
    display_next_image()

# Event handler for the Remove button
def remove_image_button(b):
    global img_idx
    if img_idx >= len(img_paths):
        print("All images have been reviewed.")
        return
    img_path, current_label, _ = img_paths[img_idx]
    remove_image(img_path, current_label)
    classification_history.append((img_path, 'Removed', None, None, current_label))  # Record the action for undo
    img_idx += 1
    completed_label.value = f"Images classified: {img_idx}"
    display_next_image()

# Button event handlers for categories
def classify_as_fire_fire(b):
    classify_image("fire", "fire")

def classify_as_fire_smoke(b):
    classify_image("fire", "smoke")

def classify_as_nofire_fire_conf(b):
    classify_image("nofire", "fire_confounders")

def classify_as_nofire_smoke_conf(b):
    classify_image("nofire", "smoke_confounders")

def classify_as_nofire_non_fire(b):
    classify_image("nofire", "non_fire")

def undo_last_classification(b):
    global img_idx
    if classification_history:
        last_img_path, last_new_label, last_category, last_subcategory, last_current_label = classification_history.pop()
        img_idx -= 1  # Go back to the last image
        img_name = os.path.basename(last_img_path)
        if last_new_label == 'Removed':
            # Move back from removed_dir to original location
            src_path = os.path.join(removed_dir, img_name)
            dst_path = last_img_path
            shutil.move(src_path, dst_path)
            with open(changes_log_path, 'a') as f:
                f.write(f"{img_name}: Undo removal\n")
            print(f"Moved {src_path} back to {dst_path} (Undo removal)")
        else:
            # Move back from corrected_dataset_dir to original location
            src_path = os.path.join(corrected_dataset_dir, last_category, last_new_label, last_subcategory, img_name)
            dst_path = last_img_path
            shutil.move(src_path, dst_path)
            with open(changes_log_path, 'a') as f:
                f.write(f"{img_name}: Undo reclassification\n")
            print(f"Moved {src_path} back to {dst_path} (Undo reclassification)")
        completed_label.value = f"Images classified: {img_idx}"
        display_next_image()  # Show the last image again
    else:
        print("No actions to undo.")

# Resume from a specific index
def set_starting_index(b):
    global img_idx
    img_idx = start_index.value
    completed_label.value = f"Images classified: {img_idx}"
    display_next_image()

# Display the next image
def display_next_image():
    global img_idx
    if img_idx >= len(img_paths):
        print("All images have been reviewed.")
        return

    img_path, current_label, _ = img_paths[img_idx]
    img_array = load_and_preprocess_image(img_path)
    heatmap = get_gradcam_heatmap(model, img_array)
    superimposed_img = superimpose_gradcam(img_path, heatmap)
    
    # Display image and Grad-CAM side by side with original label in the title
    with out:
        clear_output(wait=True)
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB))
        plt.title(f"Original Image - {current_label.capitalize()}")
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
        plt.title("Grad-CAM Visualization")
        plt.axis('off')
        plt.show()


In [5]:
# Initialize UI components
fire_fire_button = Button(description="Fire - Fire", button_style='danger')
fire_smoke_button = Button(description="Fire - Smoke", button_style='danger')
nofire_fire_conf_button = Button(description="No Fire - Fire Confounders", button_style='success')
nofire_smoke_conf_button = Button(description="No Fire - Smoke Confounders", button_style='success')
nofire_nonfire_button = Button(description="No Fire - Non Fire", button_style='success')
remove_button = Button(description="Remove", button_style='danger')
undo_button = Button(description="Undo", button_style='warning')

# Start index input
start_index = IntText(value=img_idx, description="Start from index:")
set_index_button = Button(description="Set Index")
set_index_button.on_click(set_starting_index)

# Button Click Events
fire_fire_button.on_click(classify_as_fire_fire)
fire_smoke_button.on_click(classify_as_fire_smoke)
nofire_fire_conf_button.on_click(classify_as_nofire_fire_conf)
nofire_smoke_conf_button.on_click(classify_as_nofire_smoke_conf)
nofire_nonfire_button.on_click(classify_as_nofire_non_fire)
remove_button.on_click(remove_image_button)
undo_button.on_click(undo_last_classification)

# Display UI
display(VBox([
    HBox([
        fire_fire_button, fire_smoke_button, nofire_fire_conf_button,
        nofire_smoke_conf_button, nofire_nonfire_button, remove_button, undo_button
    ]),
    completed_label,
    HBox([start_index, set_index_button]),
    out
]))

# Display the first image
display_next_image()

Moved Datasets/the_wildfire_dataset_2n_version/train/fire/26943717707_d2f58089f8_o.jpg to Datasets/corrected_wildfire_dataset/train/fire/smoke/26943717707_d2f58089f8_o.jpg


Moved Datasets/the_wildfire_dataset_2n_version/train/fire/50246118707_cfe8f84230_o.jpg to Datasets/corrected_wildfire_dataset/train/fire/smoke/50246118707_cfe8f84230_o.jpg


Moved Datasets/the_wildfire_dataset_2n_version/train/fire/27900338542_810a1ba4be_o.jpg to Datasets/corrected_wildfire_dataset/train/fire/smoke/27900338542_810a1ba4be_o.jpg


Moved Datasets/the_wildfire_dataset_2n_version/train/fire/52104987842_dc55d3abf0_o.jpg to Datasets/removed/52104987842_dc55d3abf0_o.jpg (Removed)


Moved Datasets/the_wildfire_dataset_2n_version/train/fire/48250131842_a7c57629ed_o.jpg to Datasets/corrected_wildfire_dataset/train/fire/smoke/48250131842_a7c57629ed_o.jpg


...

Moved Datasets/the_wildfire_dataset_2n_version/train/nofire/pat-whelen-FEsS8cYQM-0-unsplash.jpg to Datasets/removed/pat-whelen-FEsS8cYQM-0-unsplash.jpg (Removed)


Moved Datasets/the_wildfire_dataset_2n_version/train/nofire/52359039480_ab7270c4f1_o.jpg to Datasets/removed/52359039480_ab7270c4f1_o.jpg (Removed)


Moved Datasets/the_wildfire_dataset_2n_version/train/nofire/yann-allegre-96moO4t_E1Y-unsplash.jpg to Datasets/removed/yann-allegre-96moO4t_E1Y-unsplash.jpg (Removed)


Moved Datasets/the_wildfire_dataset_2n_version/train/nofire/yang-song-8zZCbPprSMk-unsplash.jpg to Datasets/removed/yang-song-8zZCbPprSMk-unsplash.jpg (Removed)


Moved Datasets/the_wildfire_dataset_2n_version/train/nofire/eberhard-grossgasteiger-zzZ8eq7pYqM-unsplash.jpg to Datasets/removed/eberhard-grossgasteiger-zzZ8eq7pYqM-unsplash.jpg (Removed)
